# Calculate Political Opinion Models

In [9]:
import logging
from os import path

## Training the Base Model

Calculate the base model (empty), that is later used as a base for training the classification models.

In [10]:
from corputil import FileCorpus
from gensim.models.word2vec import Word2Vec

file1 = path.join('data', 'Corpus_SPD_Fraktion.txt')
file2 = path.join('data', 'Corpus_Grüne_Fraktion.txt')
file3 = path.join('data', 'Corpus_Linke_Fraktion.txt')

base_corpus = FileCorpus(file1, file2, file3)

base = Word2Vec(workers=4, iter=4)
base.build_vocab(base_corpus.sentences_token())

Save model to disk. Don't finalize the model because we need to train it with new data later!

In [11]:
# base.save(path.join('models', 'base.w2v'))

## Training

In [12]:
from copy import deepcopy

corpora = [FileCorpus(file1), FileCorpus(file2), FileCorpus(file3)]
models = [deepcopy(base) for i in range(len(corpora))]

for i in range(len(corpora)):
    sentences = list(corpora[i].sentences_token())
    models[i].train(sentences, total_examples=len(sentences))
    print('Loaded corpus with {} sentences.'.format(len(sentences)))

Loaded corpus with 98697 sentences.
Loaded corpus with 119226 sentences.
Loaded corpus with 139882 sentences.


## Classification

In [13]:
import pandas as pd
import numpy as np

def calc_probability(docs, mods):
    sentlist = [s for d in docs for s in d]
    llhd = np.array( [ m.score(sentlist, len(sentlist)) for m in mods ] )
    lhd = np.exp(llhd - llhd.max(axis=0))
    prob = pd.DataFrame( (lhd/lhd.sum(axis=0)).transpose() )
    prob["doc"] = [i for i,d in enumerate(docs) for s in d]
    prob = prob.groupby("doc").mean()
    return prob

test_corpus = list(FileCorpus(path.join('data', 'Test.txt')).doc_sentences_token())
prob = calc_probability(test_corpus, models)
print(prob)

                0         1         2
doc                                  
0    1.642097e-01  0.210489  0.625301
1    2.542731e-02  0.352747  0.621826
2    2.591371e-09  0.497494  0.502506
3    1.551192e-06  0.411207  0.588791
4    7.153789e-03  0.003400  0.989446


In [14]:
documents = FileCorpus(path.join('data', 'Test.txt'))
for doc in documents:
    print(doc)

Brüssel kommt nicht zur Ruhe: Bei einer Anti-Terror-Razzia am späten Sonntagabend hat die Polizei 16 Personen festgenommen. Der mutmaßliche Komplize der Attentäter von Paris, Abdeslam, ist nach Angaben der Staatsanwaltschaft nicht darunter.   Die belgische Polizei hat bei Razzien in Brüssel und Umgebung am späten Sonntagabend 16 Personen festgenommen. Das teilte die Staatsanwaltschaft bei einer nächtlichen Pressekonferenz mit. 19 Häuser seien durchsucht worden, Waffen oder Sprengstoff seien nicht sichergestellt worden. Der mit internationalem Haftbefehl gesuchte Salah Abdeslam sei nicht unter den Festgenommenen. Vermutlich war er an den Pariser Attentaten mit insgesamt 130 Toten beteiligt. Er ist der Bruder eines der Selbstmordattentäter. Zur Identität der Festgenommenen wurden keine Angaben gemacht.  In der Gemeinde Molenbeek, wo bereits kurz nach den Anschlägen von Paris Razzien durchgeführt worden waren, feuerten Polizisten den Angaben zufolge auf ein Fahrzeug, das auf sie zuraste. 